In [ ]:
from psfv import *
# for documentation, open psfv/docs/build/html/index.html

In [ ]:
#choose your favorite star.
star_id = 'GAIA DR3 4690516677131714432' 
star_info = get_star_info(star_id) #searching some info in online catalogues. Results are stored, so it will run quicker the second time.
print(star_info.keys())

In [ ]:
#Check in which TESS sectors, this star has been observed
print(star_info['observed_sectors'])

In [ ]:
# choose a sector
sector=67
#download the TargetPixelFile
coordinates = str(star_info['ra'])+' '+str(star_info['dec'])
tpf = read_tpf(star_id,sector,coord=coordinates) #will be downloaded when ran for the first time.

In [ ]:
fancy_tpf_plot(tpf,star_id,save_fig=False)

#see quickt_tpf_plot() for a simpler and quicker version

In [ ]:
#Let's start with a Simple Aperature Photometry light curve
time,flux_sap = get_raw_sap_lc(star_id, sector, mask_type='3x3',save_lc=True)
fig = plt.figure(figsize=(7,3))
plt.title(star_id)
plt.plot(time,flux_sap,c='black')

plt.xlabel('Time - 2457000 [BTJD days]',fontsize=8)
plt.ylabel('SAP flux (e/s)',fontsize=8)
plt.show()

In [ ]:
#let's choose which stars to include.
#fit_input is just a collection of information, easy to pass along other methods.
fit_input = create_fit_input(star_id, 
            sector=sector, 
            max_Tmag=12, #all stars below x Tmag are ignored
            radius_inculded=3, #all stars further than x pixellengths are ignored 
            cutoutsize=11, #decide on a smaller subimage, to speed things up a bit, pixels outside the cutout are ignored for PSF fit, but included in the background estimation
            )

check_fit_input_plot(fit_input) #visual check, see documentation for more parameter options
#red circles are FWHM

In [ ]:
#once you are happy with how it looks, you can perform the fit for all cadances
fit_results = get_psf_fit_results(fit_input,overwrite=False)
 
plot_psf_fitted_fluxes(fit_results) #This lightcurve shows the targets PSF flux, i.e. the "volume" under the fitted gaussian.

In [ ]:
#The blue lightcurve above shows the targets and neigbour PSF flux, i.e. the "volume" under their fitted gaussians.
#This information can also be accessed as follows:
    
n_cad = len(fit_results['fit_results'])
k = 0 # 0 is the target, higher integers for the included neigbours, these indices correspond with what is plotted in the check_fit_input_plot, increasing with apparent distance.
flux_psf = [fit_results['fit_results'][i]['flux_fit'][k] for i in range(n_cad)]

fig = plt.figure(figsize=(7,3))
plt.title(star_id)
plt.plot(time,flux_sap/np.median(flux_sap),c='black',label='SAP flux')
plt.plot(time,flux_psf/np.median(flux_psf),c='C0',label='PSF flux')

plt.xlabel('Time - 2457000 [BTJD days]',fontsize=8)
plt.ylabel('Normalised flux',fontsize=8)
plt.legend()
plt.show()

In [ ]:
#Check out documentation for many more methods, plots and options.